In [ ]:
# Load Data and Imports
import pandas as pd
from dash import Dash, html, dcc
import plotly.express as px
import dash_ag_grid as dag

# Incorporate data
df = pd.read_excel(r"D:\_University\Fall 2025\CMPT450-TermProject-Dash\cleaned_games.csv") # Change this based on who is running the code


In [4]:
# Handling Items

df['Year released'] = pd.to_datetime(df['Release date'], errors='coerce').dt.year

# Count how many rows have invalid or missing release dates
invalid_dates_count = df['Year released'].isna().sum()
print(f"Number of rows with invalid or missing release dates: {invalid_dates_count}")

df.dropna(subset=['Year released'], inplace=True)
df['Year released'] = df['Year released'].astype(int)

# Handle Estimated Owners ===================
def parse_owners(owner_str):
    if isinstance(owner_str, str):
        try:
            low, high = owner_str.split(' - ')
            return (int(low) + int(high)) / 2
        except (ValueError, AttributeError):
            return None
    return None

df['Owners (numeric)'] = df['Estimated owners'].apply(parse_owners)

# Positive Review Percentage ====================
df['Positive Review %'] = (df['Positive'] / (df['Positive'] + df['Negative'] + 1)) * 100

# Drop rows with 0 positive reviews for analysis
df = df[df['Positive'] > 0]

# Drop Rows with 0 negative reviews to avoid skewed percentages
df = df[df['Negative'] > 0]

df['total_reviews'] = df['Positive'] + df['Negative']
df = df[df.total_reviews > 0]

df_handled = df.copy()

Number of rows with invalid or missing release dates: 23


In [ ]:
# Active Players Over Time Line Chart

active_player_frame = df[['Release date', 'Peak CCU']].copy()

# Take Release Date and Peak CCU Columns, average by Release Date Year
active_player_frame['Release date'] = pd.to_datetime(active_player_frame['Release date'], errors='coerce')
active_player_frame['Release Year'] = active_player_frame['Release date'].dt.year

# If NA in Peak CCU, drop row
active_player_frame = active_player_frame.dropna(subset=['Peak CCU'])

active_player_frame['Peak CCU'] = active_player_frame['Peak CCU'].astype(int)
active_player_frame = active_player_frame.groupby('Release Year')['Peak CCU'].mean().reset_index()

active_players_fig = px.line(
    active_player_frame,
    x='Release Year',
    y='Peak CCU',
    title='Average Active Players Over Time',
    labels={'Release Year': 'Release Year', 'Peak CCU': 'Average Peak Concurrent Users'}
)

active_players_fig.show()

In [7]:
# Genre popularity by release count over time

df_genre_analysis = df.copy()

df_genre_analysis.dropna(subset=['Genres'], inplace=True)

# --- Graph 1A: Genre Proliferation Over Time (Efficient Method) ---

# Group by year, then apply a function to count genres for that year's subset.
genre_counts_by_year = df_genre_analysis.groupby('Year released')['Genres'].apply(
    lambda x: x.str.split(',').explode().value_counts()
).reset_index()
genre_counts_by_year.columns = ['Year released', 'Genre', 'Count']

# Find the top 10 genres overall to keep the chart clean.
total_genre_counts = genre_counts_by_year.groupby('Genre')['Count'].sum()
top_10_genres = total_genre_counts.nlargest(10).index

# Filter the yearly data to only include these top genres.
df_top_genres_yearly = genre_counts_by_year[genre_counts_by_year['Genre'].isin(top_10_genres)]

df_top_genres = df_genre_analysis[df_genre_analysis['Genres'].isin(top_10_genres)]
# Group by year and genre, then count the number of games
genre_counts_by_year = df_top_genres.groupby(['Year released', 'Genres']).size().reset_index(name='count')


genre_popularity_fig = px.line(
    genre_counts_by_year,
    x='Year released',
    y='count',
    color='Genres',
    title='Number of Game Releases Per Genre Over Time',
    labels={'Year released': 'Year of Release', 'count': 'Number of Games Released'},
    markers=True,
    hover_name='Genres'
)

genre_popularity_fig.show()

In [ ]:
# Average Playtime vs Postive Review % Scatterplot


bubble_fig = px.scatter(
    df, 
    x='Average playtime forever', 
    y='Positive Review %', 
    size='count', 
    color='Genres', 
    hover_name='Genres', 
    title='Average Playtime vs Positive Review % per Genre',
    size_max=60)
# need to change the values on this because everything is 1 so there is no size 
# difference but good enough for now surely

bubble_fig.show()